In [1]:
import pandas as pd

In [16]:
df = pd.read_csv('./data/CPU.csv')

In [3]:
# Prices are in Indian Rupee and in String format, change it to USD and int using API
import requests


class CurrencyExchanger:
    rates = {}

    def __init__(self, url, headers, params):
        response = requests.get(url, headers=headers, params=params)
        data = response.json()
        self.rates = data['rates']

    def converter(self, params, amount):
        exchange_rate = self.rates[params['symbols']]
        return round(exchange_rate * amount, 2)

In [17]:
# Instancing the class object

url = 'https://api.apilayer.com/fixer/latest'
headers = {
    'apikey': 'i0khQ80cJeZhfSlcUVClOsXNtOOize60'
}
# We want to convert Indian Rupee to USA Dollar
params = {
    'base': 'INR',
    'symbols': 'USD'
}

currency_exchanger = CurrencyExchanger(url, headers, params)

#Store only the rate and use it onward to data conversion (Note: we only have 1000 request/month)
convert_rate = currency_exchanger.rates[params['symbols']]

In [18]:
# The smallest price is 1000 INR with a comma separator
min(df['MRP'])

'₹1,015'

In [19]:
# Making the price integer
df['MRP'] = df['MRP'].str.replace('₹', '')
df['MRP'] = df['MRP'].str.replace(',', '')
df['MRP'] = df['MRP'].astype(int)

In [20]:
# Price conversion to USD
df['MRP'] = round(df['MRP'] * convert_rate, 2)

In [21]:
df

,Unnamed: 0,CPU,MRP
0,0,amd Ryzen 5 3600 with Wraith Stealth Cooler (1...,117.82
1,1,amd Ryzen 9 5900X 3.7 GHz Upto 4.8 GHz AM4 Soc...,419.45
2,2,processsor Ultra 3.5 GHz LGA 1150 Intel Core i...,9.14
3,3,GIGASTAR 3.4 GHz LGA 1155 Intel i5-3570K For H...,22.72
4,4,Intel i5-12400F 4.4 GHz Upto 4.4 GHz LGA1700 S...,156.38
...,...,...,...
1435,1435,srigs 3.1 GHz LGA 1155 Intel Core i5-2400 (2nd...,15.63
1436,1436,"Bestor Dual-Sided Multifunctional Desk Pad ,Wa...",9.24
1437,1437,"Astound VGA to HDMI Converter, converts VGA & ...",12.01
1438,1438,suryaasys 3.2 GHz LGA 1156 I5 650 Processor,9.74


In [22]:
# Preparing sql export
df.rename(columns={'Unnamed: 0': 'id', 'CPU': 'name', 'MRP': 'mrp_usd'}, inplace=True)

In [47]:
# Data validation check
df = df.drop_duplicates(subset=['name', 'mrp_usd'])
df = df[df['name'].str.contains('Processor')]

In [48]:
df['architecture'] = df['name'].apply(lambda x: 'AMD' if 'amd' in x else 'INTEL')

In [49]:
df

,id,name,mrp_usd,architecture
0,0,amd Ryzen 5 3600 with Wraith Stealth Cooler (1...,117.82,AMD
1,1,amd Ryzen 9 5900X 3.7 GHz Upto 4.8 GHz AM4 Soc...,419.45,AMD
2,2,processsor Ultra 3.5 GHz LGA 1150 Intel Core i...,9.14,INTEL
3,3,GIGASTAR 3.4 GHz LGA 1155 Intel i5-3570K For H...,22.72,INTEL
4,4,Intel i5-12400F 4.4 GHz Upto 4.4 GHz LGA1700 S...,156.38,INTEL
...,...,...,...,...
1433,1433,ryzen 3.9 GHz AM4 5600G Processor,180.32,INTEL
1434,1434,amd Ryzen Processor 7 series 3.7 GHz AM4 Socke...,336.60,AMD
1435,1435,srigs 3.1 GHz LGA 1155 Intel Core i5-2400 (2nd...,15.63,INTEL
1438,1438,suryaasys 3.2 GHz LGA 1156 I5 650 Processor,9.74,INTEL


In [50]:
#sql export
from sqlalchemy import create_engine

#Placeholder localhost login
mysql_config = {
    'user': 'pycharm',
    'password': 'testtest-123',
    'host': 'localhost',
    'database': 'projectcpu'
}

engine = create_engine(
    f"mysql+mysqlconnector://{mysql_config['user']}:{mysql_config['password']}@{mysql_config['host']}/{mysql_config['database']}")

#Use only third of the data atm
df.to_sql(name='cpu', con=engine, if_exists='append', index=False)



267

In [51]:
#quick test query
from sqlalchemy import text
connection = engine.connect()

query = text("select * from cpu")
result = connection.execute(query)

counter = 0
for row in result:
    counter+=1
    print(row)
    
print(counter)
result.close()
connection.close()

(0, 'amd Ryzen 5 3600 with Wraith Stealth Cooler (100-000000031) 3.6 Ghz Upto 4.2 GHz AM4 Socket 6 Cores 12 Threads 3 MB L2 32 MB L3 Desktop Processor', 117.82, 'AMD')
(1, 'amd Ryzen 9 5900X 3.7 GHz Upto 4.8 GHz AM4 Socket 12 Cores 24 Threads Desktop Processor', 419.45, 'AMD')
(2, 'processsor Ultra 3.5 GHz LGA 1150 Intel Core i3 4150 3.5Ghz 4th generation Processor', 9.14, 'INTEL')
(3, 'GIGASTAR 3.4 GHz LGA 1155 Intel i5-3570K For H61 Motherboard 3rd Generation Processor', 22.72, 'INTEL')
(4, 'Intel i5-12400F 4.4 GHz Upto 4.4 GHz LGA1700 Socket 6 Cores 12 Threads Desktop Processor', 156.38, 'INTEL')
(5, 'amd Ryzen 5 5600G 3.9 GHz Upto 4.4 GHz AM4 Socket 6 Cores 12 Threads 3 kB L2 16 kB L3 Desktop Processor', 150.26, 'AMD')
(6, 'betaohm 3.4 GHz LGA 1155 Intel Core i5 3570 3rd Generation Processor Excellent Performance Processor Processor', 19.6, 'INTEL')
(7, 'amd Ryzen 5 5600X 3.7 GHz Upto 4.6 GHz AM4 Socket 6 Cores 12 Threads Desktop Processor', 199.55, 'AMD')
(8, 'GIGASTAR 3.4 GHz LGA